In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import pandas as pd

## NEED TO SET UP meas_modelfit *before starting ipython* for code at bottom (not included in lsst_apps!)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import diffimTests as dit
reload(dit)

testObj = dit.DiffimTest(imSize=(256, 256),
                         varFlux2=np.repeat(5000, 10), 
                         #varFlux2=np.repeat(620*np.sqrt(2), 20),
                         n_sources=600/4, verbose=True, 
                         sourceFluxRange=(2000., 250000.), saturation=250000., 
                         #bad_columns=[255, 256, 257],
                         variablesAvoidBorder=3.5, psf_yvary_factor=0., psfSize=13)
res = testObj.runTest(spatialKernelOrder=2)
print res

Template PSF: [1.6, 1.6] 0.0
Science PSF: [1.8, 2.2] -45.0
0.824621125124
Offset: [0.0, 0.0]
Flux range: 246642.393478 2000.0
PSF y spatial-variation: 0.0 0.0
Variable source: 14 196.396399934 191.955509328 0 5000
Variable source: 0 206.917338547 114.54895376 0 5000
Variable source: 11 56.0101022569 161.464859606 0 5000
Variable source: 9 109.86163575 51.8176374461 0 5000
Variable source: 21 172.232208604 192.272549224 0 5000
Variable source: 15 85.9554656804 192.742268735 0 5000
Variable source: 24 82.2011229567 68.8410845187 0 5000
Variable source: 12 137.122266426 199.536441967 0 5000
Variable source: 26 180.413834482 119.971015294 0 5000
Variable source: 22 163.555013436 128.358309666 0 5000
0 saturated pixels (template)
0 saturated pixels (science)


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. lsst.afw.detection._footprint.Footprint(nspan: int=0L, region: lsst.afw.geom.box.Box2I=Box2D(minimum=Point2I(0, 0), dimensions=Extent2I(0, 0)))
    2. lsst.afw.detection._footprint.Footprint(peakSchema: lsst.afw.table.schema.schema.Schema, nspan: int=0L, region: lsst.afw.geom.box.Box2I=Box2D(minimum=Point2I(0, 0), dimensions=Extent2I(0, 0)))
    3. lsst.afw.detection._footprint.Footprint(bbox: lsst.afw.geom.box.Box2I, region: lsst.afw.geom.box.Box2I=Box2D(minimum=Point2I(0, 0), dimensions=Extent2I(0, 0)))
    4. lsst.afw.detection._footprint.Footprint(center: lsst.afw.geom.coordinates.coordinates.Point2I, radius: float, region: lsst.afw.geom.box.Box2I=Box2D(minimum=Point2I(0, 0), dimensions=Extent2I(0, 0)))
    5. lsst.afw.detection._footprint.Footprint(ellipse: lsst.afw.geom.ellipses._ellipse.Ellipse, region: lsst.afw.geom.box.Box2I=Box2D(minimum=Point2I(0, 0), dimensions=Exte

In [ ]:
import lsst.pex.config as pexConfig
import lsst.afw.image as afwImage
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath
import lsst.meas.algorithms as measAlg
import lsst.pipe.base as pipeBase
import lsst.ip.diffim as ipDiffim

In [ ]:
exposure = testObj.im2.asAfwExposure()
template = testObj.im1.asAfwExposure()

In [ ]:
exposure2 = testObj.im2.asAfwExposure()
obj = dit.tasks.PsfMeasurement(exposure2)
obj.run()
psf = obj.exposure.getPsf()
exposure2.setPsf(psf)

template2 = testObj.im1.asAfwExposure()
obj = dit.tasks.PsfMeasurement(template2)
obj.run()
psf = obj.exposure.getPsf()
template2.setPsf(psf)

In [ ]:
def ga(exposure):
    return exposure.getMaskedImage().getImage().getArray()
def gv(exposure):
    return exposure.getMaskedImage().getVariance().getArray()
def gm(exposure):
    return exposure.getMaskedImage().getMask().getArray()

Test `computeDiffim`.

In [ ]:
config = dit.ZogyConfig()
task = dit.ZogyTask(templateExposure=template, scienceExposure=exposure, config=config)

In [ ]:
D = task.computeDiffim(inImageSpace=False)
print dit.computeClippedImageStats(ga(D))
print dit.computeClippedImageStats(gv(D))
dit.plotImageGrid((ga(D), gv(D)), imScale=4)

In [ ]:
D = task.computeDiffim(inImageSpace=True)
print dit.computeClippedImageStats(ga(D))
print dit.computeClippedImageStats(gv(D))
dit.plotImageGrid((ga(D), gv(D)), imScale=4)

In [ ]:
%timeit task.computeDiffim(inImageSpace=False)

In [ ]:
%timeit task.computeDiffim(inImageSpace=True)

Test `computeScorr`.

In [ ]:
config = dit.ZogyConfig()
task = dit.ZogyTask(templateExposure=template, scienceExposure=exposure, config=config)

S = task.computeScorr(inImageSpace=False)
print dit.computeClippedImageStats(ga(S))
print dit.computeClippedImageStats(gv(S))
dit.plotImageGrid((ga(S), gv(S), (ga(S)/gv(S) > 5.0)), imScale=4)

In [ ]:
config = dit.ZogyConfig()
task = dit.ZogyTask(templateExposure=template, scienceExposure=exposure, config=config)

SR = task.computeScorr(inImageSpace=False)
S = task.computeScorr(inImageSpace=True)
print dit.computeClippedImageStats(ga(SR))
print dit.computeClippedImageStats(gv(SR))
print dit.computeClippedImageStats(ga(S))
print dit.computeClippedImageStats(gv(S))
print dit.computeClippedImageStats(ga(SR)/ga(S))
print dit.computeClippedImageStats(gv(SR)/gv(S))

dit.plotImageGrid((ga(S), gv(S), (ga(S)/gv(S) > 5.0)), imScale=4)

Test `computeScorr` with nonzero `x/yVarAst`

In [ ]:
S = task.computeScorr(inImageSpace=False, xVarAst=0.1, yVarAst=0.1)
dit.plotImageGrid((ga(S), gv(S), (ga(S)/gv(S) > 5.0)), imScale=4)

In [ ]:
S = task.computeScorr(inImageSpace=True, xVarAst=0.1, yVarAst=0.1)
dit.plotImageGrid((ga(S), gv(S), (ga(S)/gv(S) > 5.0)), imScale=4)

Test imageMapper.

In [ ]:
config = dit.zogyTask.ZogyMapReduceConfig()
config.gridStepX = config.gridStepY = 9
#config.gridSizeX = config.gridSizeY = 10
config.borderSizeX = config.borderSizeY = 3
config.reducerSubtask.reduceOperation = 'average'
task = dit.ImageMapReduceTask(config=config)
print config

In [ ]:
DF = task.run(exposure, template=template, inImageSpace=False,
             doScorr=False, forceEvenSized=True).exposure
dit.plotImageGrid((ga(DF), gv(DF)), imScale=4)

In [ ]:
SF = task.run(exposure, template=template, inImageSpace=False,
              doScorr=True, forceEvenSized=True).exposure
dit.plotImageGrid((ga(SF), gv(SF), (ga(SF)/gv(SF) > 5.0)), imScale=4)

In [ ]:
SF = task.run(exposure, template=template, inImageSpace=False,
              doScorr=True, forceEvenSized=True, xVarAst=0.1, yVarAst=0.1).exposure
dit.plotImageGrid((ga(SF), gv(SF), (ga(SF)/gv(SF) > 5.0)), imScale=4)

In [ ]:
config = dit.zogyTask.ZogyMapReduceConfig()
config.gridStepX = config.gridStepY = 8
#config.gridSizeX = config.gridSizeY = 10
config.borderSizeX = config.borderSizeY = 6
config.reducerSubtask.reduceOperation = 'average'
task = dit.ImageMapReduceTask(config=config)
print config

In [ ]:
DR = task.run(exposure, template=template, inImageSpace=True,
              doScorr=False, forceEvenSized=True).exposure
dit.plotImageGrid((ga(DR), gv(DR)), imScale=4)

In [ ]:
SR = task.run(exposure, template=template, inImageSpace=True,
                       doScorr=True, forceEvenSized=True).exposure
dit.plotImageGrid((ga(SR), gv(SR), (ga(SR)/gv(SR) > 5.0)), imScale=4)

In [ ]:
print type(DR.getPsf()), type(SR.getPsf())

In [ ]:
dit.plotImageGrid((ga(DF) - ga(DR),), imScale=6.)

Compare masks between image/fourier calculations.

In [ ]:
config = dit.ZogyConfig()
task = dit.ZogyTask(templateExposure=template, scienceExposure=exposure, config=config)

D_F = task.computeDiffim(inImageSpace=False)
print dit.computeClippedImageStats(ga(D_F))
print dit.computeClippedImageStats(gv(D_F))

D_R = task.computeDiffim(inImageSpace=True)
print dit.computeClippedImageStats(ga(D_R))
print dit.computeClippedImageStats(gv(D_R))

In [ ]:
dit.plotImageGrid((gm(D_F), gm(D_R)), imScale=4)

In [ ]:
dit.plotImageGrid((ga(D_F) - ga(D_R), gv(D_F) - gv(D_R),), imScale=8)

In [ ]:
config = dit.ZogyConfig()
task = dit.ZogyTask(templateExposure=template, scienceExposure=exposure, config=config)

S_F = task.computeScorr(inImageSpace=False)
print dit.computeClippedImageStats(ga(D_F))
print dit.computeClippedImageStats(gv(D_F))

S_R = task.computeScorr(inImageSpace=True)
print dit.computeClippedImageStats(ga(D_R))
print dit.computeClippedImageStats(gv(D_R))

dit.plotImageGrid((ga(S_F) - ga(S_R),), imScale=8)

Debug the unit tests...

In [ ]:
import testImageDecorrelation as testID

svar = tvar = 100.
psf1_sigma = 3.3  # sigma of psf of science image
psf2_sigma = 2.2  # sigma of psf of template image
varyPsf = False

im1ex, im2ex \
    = testID.makeFakeImages(svar=svar, tvar=tvar,
                            psf1=psf1_sigma, psf2=psf2_sigma,
                            n_sources=10, psf_yvary_factor=varyPsf,
                            seed=666, verbose=False)
# Create an array corresponding to the "expected" subtraction (noise only)
np.random.seed(666)
expectedSubtraction = np.random.normal(scale=np.sqrt(svar), size=im1ex.getDimensions())  # variance of science image
expectedSubtraction -= np.random.normal(scale=np.sqrt(tvar), size=im2ex.getDimensions())  # variance of template image
dit.plotImageGrid((ga(im1ex), ga(im2ex), expectedSubtraction), imScale=4)

In [ ]:
config = dit.ZogyConfig()
task = dit.ZogyTask(templateExposure=im2ex, scienceExposure=im1ex, config=config)

DF = task.computeDiffim(inImageSpace=False)
DR = task.computeDiffim(inImageSpace=True)
print dit.computeClippedImageStats(ga(DF))
print dit.computeClippedImageStats(ga(DR))
print dit.computeClippedImageStats(expectedSubtraction)
print dit.computeClippedImageStats(ga(DF)-ga(DR))
print dit.computeClippedImageStats(ga(DF)-expectedSubtraction)
print dit.computeClippedImageStats(ga(DR)-expectedSubtraction)
dit.plotImageGrid((ga(DF), ga(DR), 
                  ga(DF)-expectedSubtraction,
                  ga(DR)-expectedSubtraction,
                  ga(DF)-ga(DR)), imScale=4.)

In [ ]:
cov, stat = dit.utils.computePixelCovariance(ga(DF), expectedSubtraction)
print stat
cov2, stat = dit.utils.computePixelCovariance(ga(DF), ga(DR))
print stat
dit.plotImageGrid((cov, cov2), imScale=4., clim=(0, 0.5), cmap=None)

In [ ]:
dit.plotImageGrid((ga(DF)[100:130,100:130], ga(DR)[100:130,100:130],
                   expectedSubtraction[100:130,100:130]), imScale=6.)